In [3]:
# ==============================================================================
# NOTEBOOK DE TESTES METAMÓRFICOS - NÍVEL 1: ANÁLISE DE SENSIBILIDADE
# ==============================================================================
# Objetivo: Avaliar a estabilidade e a sensibilidade do sistema a perturbações,
# ausências e valores extremos nos dados de entrada.
# ------------------------------------------------------------------------------

# 📦 Imports
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from pathlib import Path
from IPython.display import display

# ------------------------------------------------------------------------------
# 📁 ETAPA 1: SETUP DO AMBIENTE DE TESTE
# ------------------------------------------------------------------------------

# Carregar dados de referência
try:
    ethanol_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Etanol] Trajeto Casa-Escola-UFRN/trackLog-2023-Feb-13_06-38-49_seg.csv")
    gasoline_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Gasolina] Trajeto Casa-Escola-UFRN/trackLog-2022-Dec-01_06-43-57_qui.csv")
    df_ethanol = pd.read_csv(ethanol_path)
    df_gasoline = pd.read_csv(gasoline_path)
    print("Arquivos de dados carregados com sucesso.")
except FileNotFoundError as e:
    print(f"ERRO: Arquivos de dados não encontrados. {e}")
    raise

# Função para preparar os dados
def preparar_dados(df):
    df_copy = df.copy()
    df_copy.columns = df_copy.columns.str.strip()
    col_map = {'Speed (OBD)(km/h)': 'Speed(OBD)(km/h)', 'Latitude ': 'Latitude', 'Longitude ': 'Longitude'}
    df_copy.rename(columns={k: v for k, v in col_map.items() if k in df_copy.columns}, inplace=True)
    if "Speed(OBD)(km/h)" in df_copy.columns:
        speeds = df_copy["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df_copy["Acceleration"] = np.diff(speeds, prepend=speeds[0])
    return df_copy

df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# Carregar os modelos
try:
    modelos = {
        "ethanol": {
            "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_ethanol_afr.pkl")),
            "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_ethanol_maf.pkl"))
        },
        "gasoline": {
            "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_gasoline_afr.pkl")),
            "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_gasoline_maf.pkl"))
        }
    }
    print("Modelos de IA carregados com sucesso.")
except FileNotFoundError as e:
    print(f"ERRO: Arquivos de modelo não encontrados. {e}")
    raise

# ------------------------------------------------------------------------------
# 🔧 ETAPA 2: DEFINIÇÃO DAS FUNÇÕES DE SUPORTE
# ------------------------------------------------------------------------------

def compute_co2(afr, maf, fuel_type):
    afr = np.array(afr)
    maf = np.array(maf)
    carbon_fraction = {"ethanol": 0.5217, "gasoline": 0.8571}
    c_fraction = carbon_fraction.get(fuel_type)
    fuel_rate = np.divide(maf, afr, out=np.zeros_like(maf, dtype=float), where=afr!=0)
    return fuel_rate * c_fraction * (44.0 / 12.0)

def encontrar_coluna(df, alternativas):
    df.columns = df.columns.str.strip()
    for alt in alternativas:
        if alt in df.columns:
            return df[alt]
    return None

def executar_teste_plotly(df_original, df_modificado, modelo_afr, modelo_maf, fuel, nome_mr):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    for feature in features:
        if feature not in df_original.columns: df_original[feature] = 0
        if feature not in df_modificado.columns: df_modificado[feature] = 0

    # --- CORREÇÃO PARA A CURVA "REAL" ---
    afr_real_col = encontrar_coluna(df_original, ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"])
    maf_real_col = encontrar_coluna(df_original, ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"])
    co2_real = compute_co2(afr_real_col.values, maf_real_col.values, fuel) if afr_real_col is not None and maf_real_col is not None else None
    
    afr_orig = modelo_afr.predict(df_original[features])
    maf_orig = modelo_maf.predict(df_original[features])
    co2_orig = compute_co2(afr_orig, maf_orig, fuel)

    afr_mod = modelo_afr.predict(df_modificado[features])
    maf_mod = modelo_maf.predict(df_modificado[features])
    co2_mod = compute_co2(afr_mod, maf_mod, fuel)
    
    diff = np.abs(co2_orig - co2_mod)
    media, maxima, rmse, std = diff.mean(), diff.max(), mean_squared_error(co2_orig, co2_mod, squared=False), diff.std()

    print(f"\n{'='*80}\n🔁 {nome_mr} ({fuel.upper()})\n{'-'*80}")
    print(f"Diferença Média: {media:.4f} | Diferença Máxima: {maxima:.4f} | RMSE: {rmse:.4f} | Desvio Padrão (Std): {std:.4f}")
    
    tabela = pd.concat([
        df_original[features].iloc[:10].add_suffix("_Orig"),
        pd.DataFrame({" ": ["→"] * 10}),
        df_modificado[features].iloc[:10].add_suffix("_Mod"),
        pd.DataFrame({" ": [" "] * 10}),
        pd.DataFrame({
            "CO₂_Real": co2_real[:10] if co2_real is not None else "N/A",
            "CO₂_Pred_Orig": co2_orig[:10],
            "CO₂_Pred_Mod": co2_mod[:10]
        })
    ], axis=1).fillna("N/A")
    display(tabela.style.set_caption(f"Comparação dos dados - {nome_mr} ({fuel.upper()})"))

    fig = go.Figure()
    if co2_real is not None:
        fig.add_trace(go.Scatter(y=co2_real[:200], mode='lines', name='Real', line=dict(dash='dot', color='gray')))
    fig.add_trace(go.Scatter(y=co2_orig[:200], mode='lines', name='Predição Original', line=dict(dash='dash', color='blue')))
    fig.add_trace(go.Scatter(y=co2_mod[:200], mode='lines', name='Predição com RM', line=dict(color='red')))
    fig.update_layout(
        title=dict(text=f"Comparação Saída Variável E (200 Amostras) - {nome_mr} - {fuel.upper()}", x=0.5),
        xaxis_title="Amostras", yaxis_title="Variável E", template="plotly_white"
    )
    fig.show()

# ------------------------------------------------------------------------------
# 🔁 ETAPA 3: DEFINIÇÃO DOS CASOS DE TESTE METAMÓRFICOS - NÍVEL 1
# ------------------------------------------------------------------------------

testes_metamorficos_n1 = {
    "CT_N1_002: [Exclusiva] - Zerar Variável A": lambda df: df.copy().assign(**{'Speed(OBD)(km/h)': 0.0}),
    "CT_N1_003: [Exclusiva] - Zerar Variável B": lambda df: df.copy().assign(Acceleration=0.0),
    "CT_N1_004: [Exclusiva] - Zerar Variável C": lambda df: df.copy().assign(Latitude=0.0),
    "CT_N1_005: [Exclusiva] - Zerar Variável D": lambda df: df.copy().assign(Longitude=0.0),
    "CT_N1_006: [Sustentável] - Adicionar Ruído na Velocidade": lambda df: df.copy().assign(**{'Speed(OBD)(km/h)': df['Speed(OBD)(km/h)'] + np.random.normal(0, 0.1, size=len(df))}),
}

# ------------------------------------------------------------------------------
# 🧲 ETAPA 4: EXECUÇÃO DOS TESTES DE NÍVEL 1
# ------------------------------------------------------------------------------

print("="*80)
print("INICIANDO EXECUÇÃO DOS TESTES DE NÍVEL 1")
print("="*80)

for nome_mr, transformacao in testes_metamorficos_n1.items():
    for combustivel, df_original in [("ethanol", df_ethanol), ("gasoline", df_gasoline)]:
        df_modificado = transformacao(df_original.copy())
        executar_teste_plotly(
            df_original=df_original,
            df_modificado=df_modificado,
            modelo_afr=modelos[combustivel]["afr"],
            modelo_maf=modelos[combustivel]["maf"],
            fuel=combustivel,
            nome_mr=nome_mr
        )

Arquivos de dados carregados com sucesso.
Modelos de IA carregados com sucesso.
INICIANDO EXECUÇÃO DOS TESTES DE NÍVEL 1

🔁 CT_N1_002: [Exclusiva] - Zerar Variável A (ETHANOL)
--------------------------------------------------------------------------------
Diferença Média: 0.3016 | Diferença Máxima: 2.2210 | RMSE: 0.4900 | Desvio Padrão (Std): 0.3862


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Pred_Orig,CO₂_Pred_Mod
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,0.000000,0.000000,,0.642839,0.742631,0.727468
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,0.000000,0.555556,,0.653249,0.771505,0.727468
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,0.000000,0.000000,,0.653249,0.771505,0.727468
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,0.000000,-0.555556,,0.679275,0.659013,0.721582
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0.000000,-0.833333,,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.000000,0.000000,,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.000000,0.000000,,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,0.000000,0.833333,,1.090483,1.137803,0.706858
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,0.000000,0.000000,,1.090483,1.137803,0.706858
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,0.000000,0.000000,,0.921315,1.137803,0.706858



🔁 CT_N1_002: [Exclusiva] - Zerar Variável A (GASOLINE)
--------------------------------------------------------------------------------
Diferença Média: 0.5511 | Diferença Máxima: 2.9554 | RMSE: 0.8179 | Desvio Padrão (Std): 0.6044


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Pred_Orig,CO₂_Pred_Mod
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,0.000000,0.000000,,1.002671,1.456781,0.965052
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,0.000000,-0.277778,,1.013360,1.016708,0.884055
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,0.000000,0.277778,,2.088720,1.456781,0.965052
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,0.000000,0.555556,,2.261889,1.996380,0.965052
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,0.000000,0.555556,,2.499195,2.400486,0.940914
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,0.000000,0.833333,,2.606089,2.400486,0.940914
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,0.000000,0.833333,,3.070012,2.148721,0.880813
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,0.000000,0.833333,,2.445747,1.470940,0.773157
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,0.000000,0.000000,,1.073221,1.588029,0.783680
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,0.000000,-0.555556,,0.562265,1.085921,0.980167



🔁 CT_N1_003: [Exclusiva] - Zerar Variável B (ETHANOL)
--------------------------------------------------------------------------------
Diferença Média: 0.0817 | Diferença Máxima: 1.4412 | RMSE: 0.1847 | Desvio Padrão (Std): 0.1657


...

,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Pred_Orig,CO₂_Pred_Mod
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,0.000000,5,0.000000,,1.002671,1.456781,0.354376
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,0.000000,4,-0.277778,,1.013360,1.016708,0.631768
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,0.000000,5,0.277778,,2.088720,1.456781,0.354376
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,0.000000,7,0.555556,,2.261889,1.996380,0.574012
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,0.000000,9,0.555556,,2.499195,2.400486,0.818026
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,0.000000,12,0.833333,,2.606089,2.400486,0.653651
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,0.000000,15,0.833333,,3.070012,2.148721,0.505126
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,0.000000,18,0.833333,,2.445747,1.470940,0.484962
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,0.000000,18,0.000000,,1.073221,1.588029,0.481107
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,0.000000,16,-0.555556,,0.562265,1.085921,0.750878



🔁 CT_N1_006: [Sustentável] - Adicionar Ruído na Velocidade (ETHANOL)
--------------------------------------------------------------------------------
Diferença Média: 0.0203 | Diferença Máxima: 0.8143 | RMSE: 0.0612 | Desvio Padrão (Std): 0.0577


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Pred_Orig,CO₂_Pred_Mod
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,2.940173,0.000000,,0.642839,0.742631,0.933717
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,4.890107,0.555556,,0.653249,0.771505,0.764202
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,5.175330,0.000000,,0.653249,0.771505,0.771505
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,2.943814,-0.555556,,0.679275,0.659013,0.659741
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0.052910,-0.833333,,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.107826,0.000000,,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.002780,0.000000,,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,3.049151,0.833333,,1.090483,1.137803,1.137803
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,2.896038,0.000000,,1.090483,1.137803,0.849757
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,3.047590,0.000000,,0.921315,1.137803,1.137803



🔁 CT_N1_006: [Sustentável] - Adicionar Ruído na Velocidade (GASOLINE)
--------------------------------------------------------------------------------
Diferença Média: 0.0161 | Diferença Máxima: 0.7365 | RMSE: 0.0540 | Desvio Padrão (Std): 0.0516


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Pred_Orig,CO₂_Pred_Mod
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,5.094597,0.000000,,1.002671,1.456781,1.456781
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,4.083986,-0.277778,,1.013360,1.016708,1.016708
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,4.901649,0.277778,,2.088720,1.456781,1.456781
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,7.127769,0.555556,,2.261889,1.996380,1.996380
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,9.047042,0.555556,,2.499195,2.400486,2.400486
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,12.005319,0.833333,,2.606089,2.400486,2.400486
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,15.123529,0.833333,,3.070012,2.148721,2.148721
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,17.846494,0.833333,,2.445747,1.470940,1.470940
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,18.077189,0.000000,,1.073221,1.588029,1.588029
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,16.081665,-0.555556,,0.562265,1.085921,1.085921
